## *Required Libraries*

In [41]:
import pandas as pd
import requests
import os
import plotly.graph_objects as go

## *Loading the dataset using Tiingo API*

In [27]:
# Set your Tiingo API key
os.environ['TIINGO_API_KEY'] = '64f3e5fce53c85958846e25e4c4d10202b11d78a'

def get_tiingo_data(ticker, start_date, end_date):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f"Token {os.getenv('TIINGO_API_KEY')}"
    }
    
    url = f"https://api.tiingo.com/tiingo/daily/{ticker}/prices"
    params = {
        'startDate': start_date,
        'endDate': end_date,
    }
    
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200:
        data = response.json()
        data = pd.DataFrame(data)
        data['date'] = pd.to_datetime(data['date'])
        data.set_index('date', inplace=True)
        return data
    else:
        print(f"Error: {response.status_code}")
        return None

# Usage
ticker = 'AAPL'
start_date = '2020-01-01'
end_date = '2024-01-01'

data = get_tiingo_data(ticker, start_date, end_date)

In [28]:
data.head()

,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
date,,,,,,,,,,,,
2020-01-02 00:00:00+00:00,300.35,300.60,295.19,296.24,33911864,72.961236,73.021966,71.707765,71.962832,135647456,0.0,1.0
2020-01-03 00:00:00+00:00,297.43,300.58,296.50,297.15,36633878,72.251907,73.017108,72.025991,72.183890,146535512,0.0,1.0
2020-01-06 00:00:00+00:00,299.80,299.96,292.75,293.79,29644644,72.827630,72.866497,71.115039,71.367676,118578576,0.0,1.0
2020-01-07 00:00:00+00:00,298.39,300.90,297.48,299.84,27877655,72.485111,73.094842,72.264053,72.837346,111510620,0.0,1.0
2020-01-08 00:00:00+00:00,303.19,304.44,297.16,297.16,33090946,73.651131,73.954782,72.186319,72.186319,132363784,0.0,1.0


In [29]:
data=data.reset_index()

In [30]:
data.head()

,date,close,high,low,open,volume,adjClose,adjHigh,adjLow,adjOpen,adjVolume,divCash,splitFactor
0,2020-01-02 00:00:00+00:00,300.35,300.60,295.19,296.24,33911864,72.961236,73.021966,71.707765,71.962832,135647456,0.0,1.0
1,2020-01-03 00:00:00+00:00,297.43,300.58,296.50,297.15,36633878,72.251907,73.017108,72.025991,72.183890,146535512,0.0,1.0
2,2020-01-06 00:00:00+00:00,299.80,299.96,292.75,293.79,29644644,72.827630,72.866497,71.115039,71.367676,118578576,0.0,1.0
3,2020-01-07 00:00:00+00:00,298.39,300.90,297.48,299.84,27877655,72.485111,73.094842,72.264053,72.837346,111510620,0.0,1.0
4,2020-01-08 00:00:00+00:00,303.19,304.44,297.16,297.16,33090946,73.651131,73.954782,72.186319,72.186319,132363784,0.0,1.0


In [32]:
data1=data['close']
data1.shape

(1006,)

## *Visualizing Stock Trends*

In [40]:

# Create the figure
fig = go.Figure()

# Add the line plot
fig.add_trace(go.Scatter(x=data1.index, y=data1.values, mode='lines', name='Stock Price'))

# Update the layout
fig.update_layout(
    title='Stock Trend',
    xaxis_title='Date',
    yaxis_title='Price',
    width=700,
    height=500
)

# Show the plot
fig.show()